In [1]:
# LEFT OFF:  Need to split the models.  1 for items with age of well.  1 without.

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

# SVM
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

# DECISION TREE
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# RANDOM FORREST
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# mac
# workdir = '/Project/data/dd_WaterTable/'
# windows
workdir = 'D:\\project\\data\\dd_watertable\\'
file_train_data = 'training_data.csv'
file_train_label = 'training_label.csv'
file_test_data = 'test_data.csv'

In [4]:
# READ IN - ALSO RESTARTING POINT BEFORE MASSAGING, FETURES, DUMMY CODES
df_train_data = pd.read_csv(workdir + file_train_data)
df_train_label = pd.read_csv(workdir + file_train_label)
df_test_data = pd.read_csv(workdir + file_test_data)

# Merge the train data with the ID
df_train_data = pd.merge(df_train_data, df_train_label, on ='id')


# COLUMN DATA TYPE MASSAGING
#  TODO: If time permits - find a more elgant way to code

#------------------
# DATE RECORDED
#------------------
df_train_data['date_recorded'] = df_train_data['date_recorded'].astype('datetime64[ns]')
df_test_data['date_recorded'] = df_test_data['date_recorded'].astype('datetime64[ns]')

#------------------
# CONSTRUCTION YEAR
#------------------
# Convert to a string - Had issues with keeping as a int (converting over to float)
# not the cleanest work-around, but works
df_train_data['construction_year'] = df_train_data['construction_year'].astype(str)

# Set this to June 1 (chnged to 1-1 becuase of negative vlues)
# TODO: Go back and make this more intelligent.
df_train_data.loc[df_train_data['construction_year'] != '0','construction_year'] = (df_train_data['construction_year'] + '-01-01') #.astype('datetime64[ns]')

# if equal to 0 set to np.nan
df_train_data.loc[df_train_data['construction_year'] == '0','construction_year'] = ''

# convert to datetime
df_train_data['construction_year'] = df_train_data['construction_year'].astype('datetime64[ns]')

# Test data
df_test_data['construction_year'] = df_test_data['construction_year'].astype(str)
df_test_data.loc[df_test_data['construction_year'] != '0','construction_year'] = (df_test_data['construction_year'] + '-01-01') #.astype('datetime64[ns]')
df_test_data.loc[df_test_data['construction_year'] == '0','construction_year'] = ''
df_test_data['construction_year'] = df_test_data['construction_year'].astype('datetime64[ns]')


## FEATURE ENGINEERING

In [5]:
#####################
# FEATURE ENGINEERING - Run first before dummy codes
#####################
from datetime import datetime

# function to determine how old the well is (months)
def func_ft_diff_month(cols):
    d1 = cols[0]
    d2 = cols[1]
    
    if d1 == np.datetime64('NaT') or d2 == np.datetime64('NaT'):
        return 0
    else:
        return (d1.year - d2.year) * 12 + d1.month - d2.month

# Build feature
df_train_data['ft_monthsSinceBuild'] = df_train_data[['date_recorded','construction_year']].apply(func_ft_diff_month,axis=1)
df_test_data['ft_monthsSinceBuild'] = df_test_data[['date_recorded','construction_year']].apply(func_ft_diff_month,axis=1)

# Averge the rest?? - 34% -- Perhaps break this out into additional features
df_train_data[df_train_data['ft_monthsSinceBuild']==np.NaN]['ft_monthsSinceBuild'] = df_train_data['ft_monthsSinceBuild'].mean()
df_test_data[df_test_data['ft_monthsSinceBuild']==np.NaN]['ft_monthsSinceBuild'] = df_test_data['ft_monthsSinceBuild'].mean()


# Fill in the months since the build with an average (or other)
# when time permits- split this out into a feature
train_ftMnths_mean = df_train_data['ft_monthsSinceBuild'].mean()
test_ftMnths_mean = df_test_data['ft_monthsSinceBuild'].mean()

df_train_data['ft_monthsSinceBuild'].replace({np.nan: train_ftMnths_mean}, inplace=True)
df_test_data['ft_monthsSinceBuild'].replace({np.nan: test_ftMnths_mean}, inplace=True)
# or replace with 12 or 0 (at least one year or nothing)
#df_train_data['ft_monthsSinceBuild'].replace({np.nan: 12}, inplace=True)
#df_test_data['ft_monthsSinceBuild'].replace({np.nan: 12}, inplace=True)

# Bucket the Population sizes
def func_ft_PopSizeEst(cols):
    popSize = cols[0].astype('int64')
    
    if popSize == 0:
        return 'ft_PopSize_Unknown'
    if (popSize >= 0 and popSize < 50):
        return 'ft_PopSize_Small'
    if (popSize >= 50 and popSize < 100):
        return 'ft_PopSize_Medium'
    if popSize > 100 and popSize < 500:
        return 'ft_PopSize_Large'
    if popSize >= 500:
        return 'ft_PopSize_ExtraLarge'
    
# Population
df_train_data['ft_PopSizeEst'] = df_train_data[['population']].apply(func_ft_PopSizeEst, axis=1)
df_test_data['ft_PopSizeEst'] = df_test_data[['population']].apply(func_ft_PopSizeEst, axis=1)

# Handle for scheme management of None, can set this to NA
#df_train_data[['scheme_management']=='None']['']
df_train_data['scheme_management'].replace({'None': np.nan}, inplace=True)

# Convert the construction year to a year, as model must use a float/num
#construction_year

# verified
#df_train_data[['population','ft_PopSizeEst']].head(3)

## CREATE DUMMY CODES

**!! The List variables in this section control which variables we are using !! **

In [6]:
# do each one in a consolidated
def convert_dummy_vars(dataFrame, columnName, dropFirst=False, colRename=''):
    
    temp = pd.get_dummies(dataFrame[columnName], drop_first=dropFirst)
    
    if colRename != '':
        temp.columns = [(colRename)]
    
    dataFrame.drop([columnName], axis=1, inplace=True)
    dataFrame = pd.concat([dataFrame, temp], axis=1)    
    return dataFrame

def fn_add_dummy_variables(df_feat_train, df_feat_test):

    # Template
    #for column in df:
    #    print(df[column])

    # Next loop through all columns to convert to dummy variables
    for column in df_feat_train:

        if column == 'ft_monthsSinceBuild' or column=='construction_year':
            # do nothing - skip
            a = 'skip'

        elif column == 'public_meeting':
            # special case
            df_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='public_meeting', dropFirst=True, colRename='public_meeting_True')
        else:
            df_feat_train = convert_dummy_vars(df_feat_train, column)

        # print(df_feat_train.shape)

    print('\n')
    print('Test Shapes')

    for column in df_feat_test:

        if column == 'ft_monthsSinceBuild' or column=='construction_year':
            # do nothing - skip
            a = 'skip'

        elif column == 'public_meeting':
            # special case
            df_feat_test = convert_dummy_vars(dataFrame=df_feat_test, columnName='public_meeting', dropFirst=True, colRename='public_meeting_True')
        else:
            df_feat_test = convert_dummy_vars(df_feat_test, column)

    print('Train Cols: ' + str(len(df_feat_train.columns)) + '  Test Cols: ' + str(len(df_feat_test.columns)))
            
    return df_feat_train, df_feat_test

In [7]:
# CHOOSE WHICH FEATURES TO INCLUDE
# 'amount_tsh', - leave out - add after binning and plotting
shortList_vTF = ['ft_monthsSinceBuild'] # 'ft_PopSizeEst',


shortList_v1 = ['public_meeting','quantity','payment','waterpoint_type','source_class']
shortList_v2 = ['public_meeting','quantity','payment','waterpoint_type','source_class','ft_PopSizeEst','ft_monthsSinceBuild']

# Need to creat columns to OMIT

# gps_height
# 'construction_year',
LongList_v1 = ['basin','district_code','public_meeting','scheme_management',
                'extraction_type_class','management','payment','quantity','source','waterpoint_type','source_class',
             'ft_monthsSinceBuild','ft_PopSizeEst']

# swap out source_class with source + add region
# 'construction_year',
LongList_v2 = ['basin','district_code','public_meeting','scheme_management',
                'extraction_type_class','management','payment','quantity','source','waterpoint_type','region', \
               'ft_monthsSinceBuild','ft_PopSizeEst']

# Add source class
# 'construction_year',
LongList_v3 = ['basin','district_code','public_meeting','scheme_management',
                'extraction_type_class','management','payment','quantity','source','waterpoint_type','source_class','region', \
               'ft_monthsSinceBuild','ft_PopSizeEst']

LongList_v4 = LongList_v3 + ['permit']

# Items to try
# construction_year - took out - would have to convert to year int, about the same becuase I am using ft_months since build



# Choose which feature list we are working with
df_feat_train = df_train_data[LongList_v2]
df_feat_test = df_test_data[LongList_v2]

# Convert to dummy variables
df_feat_train, df_feat_test = fn_add_dummy_variables(df_feat_train, df_feat_test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':




Test Shapes
Train Cols: 116  Test Cols: 116


In [ ]:
df_feat_train.head(5)

In [ ]:
df_feat_test.head(5)

In [ ]:
# df_feat_train.rename(columns={"True":'public_meeting'}, inplace=True)

In [ ]:
# df_feat_train.head()

In [ ]:
#df_feat_train.head(3)

In [ ]:
# drop the existing columns
# df_feat_train.drop(['public_meeting','quantity','payment','waterpoint_type','source_class'], axis=1, inplace=True)
# df_feat_train = pd.concat([df_feat_train,public_meeting, quantity, payment, waterpoint_type, source_class], axis=1)

# df_feat_test.drop(['public_meeting','quantity','payment','waterpoint_type','source_class'], axis=1, inplace=True)
# df_feat_test = pd.concat([df_feat_test,test_public_meeting, test_quantity, test_payment, test_waterpoint_type, test_source_class], axis=1)


In [ ]:
#df_feat_train.tail(5)

In [ ]:
#df_feat_test.tail(5)

## SVM MODEL

In [ ]:
# Use a SVM
runSVMModel = False

In [ ]:
##############
# LOCAL SPLIT
##############

if runSVMModel == True:
    X = df_feat_train
    y = df_train_label['status_group']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=101)

    model = SVC()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Print Predictions
    print(confusion_matrix(y_test, predictions))
    print('\n')
    print(classification_report(y_test, predictions))

In [ ]:
######################################
# let's try with a different parameter
######################################

if runSVMModel == True:
    bParamSearch = 0

    if (bParamSearch == 1):
        param_grid = {'C':[0.1,1,10,100,1000], 'gamma':[1,0.1,0.01,0.001,0.0001]}
        grid = GridSearchCV(SVC(), param_grid, verbose=3)
        grid.fit(X_train, y_train)

        # best parameter
        # {'C': 0.1, 'gamma': 1}
        grid.best_params_

        grid.best_estimator_
        grid_predictions = grid.predict(X_test)

    # Recheck everything
    if (bParamSearch == 1):
        print(confusion_matrix(y_test, grid_predictions))
        print('\n')
        print(classification_report(y_test, grid_predictions))

In [ ]:
####################################
# WITH FULL RUN 
####################################

if runSVMModel == True:
    # X = df_feat_train
    # y = df_train_label['status_group']
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=101)

    X_train = df_feat_train
    y_train = df_train_label['status_group']
    X_test = df_feat_test

In [ ]:
# Find the best parameter - if we flag to do so
if runSVMModel == True:
    bFindProdParmeter = 0
    if bFindProdParmeter == 1:
        param_grid = {'C':[0.1,1,10,100,1000], 'gamma':[1,0.1,0.01,0.001,0.0001]}
        grid = GridSearchCV(SVC(), param_grid, verbose=3,n_jobs=6)
        grid.fit(X_train, y_train)

In [ ]:
# if we are seraching for best parameter, then print it (to keep the verbose logging above separate)

if runSVMModel == True:
    if bFindProdParmeter == 1:
        print(grid.best_params_)
        # {'C': 0.1, 'gamma': 1}
        # {'gamma': 0.1, 'C': 10}  - shortlist2
        print(grid.best_estimator_)

In [ ]:
if runSVMModel == True:
    # shortlist
    # {'C': 0.1, 'gamma': 1}
    # .7162

    # shortlist2
    # {'gamma': 0.1, 'C': 10}  

    # start tracking Time
    print('Starting Model Run')
    startTime = datetime.now()

    ###################################
    # RUN THE MODEL
    model = SVC(C=10, gamma=0.1)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    ###################################

    print ('Time to Finish: ' + str(datetime.now() - startTime)) 

    df_predictions = pd.DataFrame(predictions)
    df_predictions.columns=['status_group']

    # TODO: Need to add a date/time stamp and record what each model does
    # df_final = pd.merge(df_predictions, df_test_data, on='index')
    df_final = pd.concat([df_predictions, df_test_data], axis=1)[['id','status_group']]
    df_final.set_index('id', inplace=True)
    export_file = workdir + 'submission.csv'
    df_final.to_csv(export_file)

## Decision Tree

In [ ]:
runDecisionTreeModel = False
if runDecisionTreeModel == True:
    # Choose which feature list we are working with
    df_feat_train = df_train_data[LongList_v4]
    df_feat_test = df_test_data[LongList_v4]

    # Convert to dummy variables
    df_feat_train, df_feat_test = fn_add_dummy_variables(df_feat_train, df_feat_test)

In [ ]:
# LongList_v3 - 78, 43, 79 + 85, 33, 72

In [ ]:
if runDecisionTreeModel == True:
    print('Starting Model Run')
    startTime = datetime.now()

    X = df_feat_train
    y = df_train_label['status_group']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=101)

    dtree = DecisionTreeClassifier()
    dtree.fit(X_train, y_train)
    predictions = dtree.predict(X_test)

    print ('Time to Finish: ' + str(datetime.now() - startTime)) 

    # Print Predictions
    print(confusion_matrix(y_test, predictions))
    print('\n')
    print(classification_report(y_test, predictions))

In [ ]:
# Run Production
if runDecisionTreeModel == True:
    X_train = df_feat_train
    y_train = df_train_label['status_group']
    X_test = df_feat_test

    dtree = DecisionTreeClassifier()
    dtree.fit(X_train, y_train)
    predictions = dtree.predict(X_test)

    df_predictions = pd.DataFrame(predictions)
    df_predictions.columns=['status_group']

    # TODO: Need to add a date/time stamp and record what each model does
    # df_final = pd.merge(df_predictions, df_test_data, on='index')
    df_final = pd.concat([df_predictions, df_test_data], axis=1)[['id','status_group']]
    df_final.set_index('id', inplace=True)
    export_file = workdir + 'submission.csv'
    df_final.to_csv(export_file)

## Random Forests

In [ ]:
runRandomForest = False

In [ ]:
if runRandomForest==True:
    # Choose which feature list we are working with
    df_feat_train = df_train_data[LongList_v4]
    df_feat_test = df_test_data[LongList_v4]

    # Convert to dummy variables
    df_feat_train, df_feat_test = fn_add_dummy_variables(df_feat_train, df_feat_test)

In [ ]:
if runRandomForest==True:
    # 78, 60, 82 - with Long_v1
    # 78, 60, 82 - with Long_v2
    # 78, 60, 82 - with Long_v3
    # 78, 62, 82 - with Long_v4

    #######
    # TEST 
    #######
    X = df_feat_train
    y = df_train_label['status_group']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=101)

    #rfc = RandomForestClassifier(n_estimators=200)
    #rfc = RandomForestClassifier(n_estimators=400, min_samples_split=10)
    rfc = RandomForestClassifier(n_estimators=400, min_samples_split=10)
    rfc.fit(X_train, y_train)
    predictions = rfc.predict(X_test)

    # Print Predictions
    print(confusion_matrix(y_test, predictions))
    print('\n')
    print(classification_report(y_test, predictions))

In [ ]:
#################
# Run Production
#################
if runRandomForest==True:
    X_train = df_feat_train
    y_train = df_train_label['status_group']
    X_test = df_feat_test

    #rfc = RandomForestClassifier(n_estimators=200)
    rfc = RandomForestClassifier(n_estimators=400, min_samples_split=10)
    rfc.fit(X_train, y_train)
    predictions = rfc.predict(X_test)

    df_predictions = pd.DataFrame(predictions)
    df_predictions.columns=['status_group']

    # TODO: Need to add a date/time stamp and record what each model does
    # df_final = pd.merge(df_predictions, df_test_data, on='index')
    df_final = pd.concat([df_predictions, df_test_data], axis=1)[['id','status_group']]
    df_final.set_index('id', inplace=True)
    export_file = workdir + 'submission.csv'
    df_final.to_csv(export_file)